## Properties / Metadata
The document properties represent the fields that define the data elements you want to retrieve as well as the elements you choose to filter your results.  In addition, properties are at the heart of other Search parameters such as Navigation, Grouping, and Boosting.  There are hundreds of properties available for each View providing critical details needed to fulfill your searching requirements.

In order for you to be successful at search, you will need to understand how to access the available properties, how to determine what values they contain and what they mean.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta4"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.20.1 Refinitiv.Data.Content, 1.0.0-beta4

Loading extensions from `C:\Users\U8007876\.nuget\packages\microsoft.data.analysis\0.20.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Delivery.Request;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

2023-02-21 4:03:53 PM:DesktopSession. (State: DesktopSession is Pending)
2023-02-21 4:03:55 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
2023-02-21 4:03:55 PM:DesktopSession. (State: DesktopSession is Opened)


#### Metadata
MetaData is a convenient service that allows you to list all available properties within a View.  To properly build a metadata list for inspection, you can issue the following:

**Note**: The RDP Libraries for .Net does not currently provide a Content Layer interface for the Search MetaData service.  This will be coming soon.  As such, the code segment below will provide the interface at the Delivery layer.

In [5]:
// Specify the view
var response = MetaData.Definition(Search.View.CommodityQuotes).GetData();
response.Data.Table

AAACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
AACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
ABSMBSBondsRIC,String,False,False,False,False,False,False
ActivityDate,Date,True,True,True,False,False,False
ACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
AdmissionToTrading,Date,True,True,True,False,False,False
AliasRIC,String,True,False,False,True,True,True
AllATMOptionsContinuationChainsRIC,String,False,False,False,False,False,False
AllStrikesOptionsChainRIC,String,False,False,False,False,False,False
AlternateISIN,String,True,False,False,True,False,True
AnalyticRIC,String,True,False,False,True,False,True


#### Debugging (*_debugall*)
While MetaData will help list all properties available within a View, all properties will not be available for use.  That is, depending on the type of search you perform, will determine which properties will be populated - not all will be.  This is where users can use a technique to list all available properties that are populated, for the specific search requested.

In [6]:
// Note: It is critical you select one document top=1 because the amount of data return is very large
var response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                       .Top(1)   
                                                       .Select("_debugall")
                                                       .GetData();
Console.WriteLine(response.Data?.Raw);

{
  "Total": 39851,
  "Hits": [
    {
      "meta": {
        "_id": "37036",
        "_index": "sip_organisation_0.0.12",
        "_score": 2838872.0,
        "_timestamp": 1677001510806,
        "_timestamp_human": "2023-02-21T17:45:10.806Z",
        "_version": 923
      },
      "raw_source": {
        "SubsidiariesBondsCount": 303,
        "RCSTRBCLeafML": "ja IT サービス & コンサルティング|zh-Hans 信息技术服务及咨询|zh-Hant 信息技術服務及諮詢",
        "DebtToAssets": 0.856092,
        "RCSOrganisationTypeGenealogy": "OT:1",
        "QuickRatio": 0.82959,
        "TotalAssetsToEquity": 6.98381,
        "AddressCity": "ARMONK",
        "LoanCount": 0,
        "RevenueInterim": 16690000000,
        "CountryHeadquarters": "USA",
        "SDCID": "3104001",
        "IssuerHasCds": true,
        "NetProfit": 5743000000,
        "UnderlyingEquityQuoteIssueCount": 0,
        "PrimaryRICCUSIP": "459200101",
        "RCSOrganisationTypeName": "Corporation",
        "MXID": "100085757",
        "RegistrationAuthorityOr

#### Understanding values using Navigators
Using the output from a debugging session, there are a number of common properties that are very useful when filtering your search.  For example, you may want to filter your results based on a specific category of data.  Using navigators will help us understand these categories.

Looking at our debug output above, one useful property: **SearchAllCategoryv3** will helps us understand all available categories of documents the query term 'IBM' was found.  To list these categories, use the following:

In [7]:
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(0)
                                                   .Navigators("SearchAllCategoryv3")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["SearchAllCategoryv3"]["Buckets"]);

[
  {
    "Label": "Other Quote",
    "Count": 21887
  },
  {
    "Label": "Warrants",
    "Count": 6806
  },
  {
    "Label": "Bond Pricing",
    "Count": 2691
  },
  {
    "Label": "Bonds",
    "Count": 2102
  },
  {
    "Label": "Commercial Papers",
    "Count": 1312
  },
  {
    "Label": "Options",
    "Count": 1198
  },
  {
    "Label": "Futures",
    "Count": 782
  },
  {
    "Label": "Companies/Issuers",
    "Count": 780
  },
  {
    "Label": "Mergers & Acquisitions",
    "Count": 743
  },
  {
    "Label": "Equities",
    "Count": 563
  },
  {
    "Label": "Credit Default Swaps",
    "Count": 292
  },
  {
    "Label": "Analyst",
    "Count": 242
  },
  {
    "Label": "Officers",
    "Count": 229
  },
  {
    "Label": "FX & Money",
    "Count": 64
  },
  {
    "Label": "Other Instrument",
    "Count": 60
  },
  {
    "Label": "Fixed Income Curves",
    "Count": 48
  },
  {
    "Label": "Funds",
    "Count": 12
  },
  {
    "Label": "Mortgage Backed Securities",
    "Count": 12
  

Using the above listing, we can narrow our result set based on these categories using a Filter.

#### Special Properties
The Search service includes a number of special properties that enhance the ability to find meaningful results. In addition to the special property *_debugall* used within the **select** parameter as outlined above, the following properties are available: 

In [8]:
// The 'code' specification will search across all codes, such as RICs, tickers, SEDOLs etc.  
// Primarily used within a filter but can be used as a selectable property.
response = Search.Definition(Search.View.SearchAll).Filter("code eq 'IBM'")
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",,37036,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, MiFID Composite Cross Market Service",21523029548,131770829,IBMIUSD.xbo
QUOTExEQUITY,"International Business Machines Corp, Depository Receipt, Bolsa de Comercio de Buenos Aires",55838283585,1006058,IBM.BA
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, New York Stock Exchange",55838323096,1090370,IBM.N
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, London Stock Exchange",55836050580,1283576,IBMI.L
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NASDAQ INTERMARKET",55835333953,765744,IBM.TH
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, Cboe EDGX Exchange",21475059739,73533774,IBM.DG
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Arca",55837129318,753753,IBM.P
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, Cboe BZX Exchange",55845629228,53703996,IBM.Z


In [9]:
// The 'name' specification will search across all names, such as common name, legal name, issuer name, etc. 
// Primarily used within a filter but can be used as a selectable property.
response = Search.Definition(Search.View.SearchAll).Filter("name eq 'IBM'")
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",,37036,
ORGANISATION,"Banco IBM SA, Private Company",,76208,
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM
ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",,18062670,
QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",21481052421,48924732,0#IBMF:
QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",21612423771,259118763,0#IBMDF:
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",21481052892,49450681,IBMFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",21481053949,50092347,IBMFc2
QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",21613372305,260213021,IBMDFc1
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",21481053950,50092348,IBMFc3


In [10]:
// The special character underscore (_) is a convenient token that can be used to select the default Properties.  
// This will allow users to quickly select the default values plus any other properties when testing out your search.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(1000)
                                                   .Select("_, SearchAllCategoryv3")
                                                   .GetData();
response.Data.Table

ORGANISATION,"International Business Machines Corp, Public Company",,37036,,Companies/Issuers
ORGANISATION,"Banco IBM SA, Private Company",,76208,,Companies/Issuers
QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM,Equities
ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",,18062670,,Companies/Issuers
QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",21481052421,48924732,0#IBMF:,Futures
QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",21612423771,259118763,0#IBMDF:,Futures
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",21481052892,49450681,IBMFc1,Futures
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",21481053949,50092347,IBMFc2,Futures
QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",21613372305,260213021,IBMDFc1,Futures
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",21481053950,50092348,IBMFc3,Futures
QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex",21481053951,50092349,IBMFc4,Futures


#### Common Properties
As you become more familiar with search, you will soon recognize there will be a number of very useful properties that will act as your "go-tos" when filtering or selecting fields.

In [11]:
// DocumentTitle
// As you've seen in some of the above output, this property is on by default when you want to display your search results.  
// The DocumentTitle property is a quick way to describe the content retrieved for each matching document.  
// It is made up of the following child properties: 
//    'DTSubjectName', 'DTSimpleType', 'DTSource', and 'DTCharacteristics'.
var select = "DocumentTitle, DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.SearchAll).Select(select)
                                                   .Query("IBM")
                                                   .Top(100)
                                                   .GetData();
response.Data.Table

"International Business Machines Corp, Public Company",International Business Machines Corp,Public Company,,
"Banco IBM SA, Private Company",Banco IBM SA,Private Company,,
"International Business Machines Corp, Ordinary Share, NYSE Consolidated",International Business Machines Corp,Ordinary Share,NYSE Consolidated,
"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4,Private Company,,
"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Chain Contract,Equity Future,Eurex,USD
"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",Euronext Amsterdam IBM Dividend Future Chain Contracts,Equity Future,Euronext Amsterdam,USD
"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 1,Equity Future,Eurex,USD
"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 2,Equity Future,Eurex,USD
"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1,Equity Future,Euronext Amsterdam,USD
"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 3,Equity Future,Eurex,USD
"Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Continuation 4,Equity Future,Eurex,USD


In [12]:
// SearchAllCategoryv3
// When trying to narrow down your search to specific categories, this property can be extremely useful.  
// When you apply a Navigator on this property, it will return a list of ~40 categories, depending on the view. 
// Categories include asset-related instruments such as:
//     'Equities', 'Futures' , 'Options', 'Bonds', 'Commodities' 
//     but can also include things like: 'People', 'Analyst', 'Officers'.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Filter("SearchAllCategoryv3 eq 'People'")
                                                   .GetData();
response.Data.Table

PERSON,John Denham - IsUnknown,34414661717,34414661717
PERSON,Charlie Maxfield - IsUnknown,34414669683,34414669683
PERSON,Claudia Barclay - IsUnknown,34414721056,34414721056
PERSON,Robert Ploder - IsUnknown,34414738667,34414738667
PERSON,Tom Cadigan - IsUnknown,34414743082,34414743082
PERSON,R. L. Jay Vivian - IsUnknown,34414808510,34414808510
PERSON,William E. Baker - IsUnknown,34415092341,34415092341


In [13]:
// CodeResolutionBase
// Represents the most significant codes defined within the document.  For example, when searching for a symbol such 
// as ISIN, SEDOL, RIC, etc, filtering against this property will significantly reduce the matches and only bring 
// back the most relevant results.
response = Search.Definition(Search.View.SearchAll).Filter("CodeResolutionBase eq 'US046353AQ14'")
                                                   .Select("DocumentTitle, ISIN, SEDOL, RIC")
                                                   .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7


In [14]:
// Conversely, if we were to choose the more generic term 'code', would result in a wider range of matches.
response = Search.Definition(Search.View.SearchAll).Filter("code eq 'US046353AQ14'")
                                                   .Select("DocumentTitle, ISIN, SEDOL, RIC")
                                                   .GetData();
response.Data.Table

"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS AXESSALL",,BF4SNM7,US046353AQ14=MKAX
"AstraZeneca PLC, Bond Quote, TRACE Eligible, TRADEWEB DIRECT",,BF4SNM7,046353AQ1=TWBL
"AstraZeneca PLC, Bond Quote, TRACE Eligible, REAL TIME ALL QUOTES",,BF4SNM7,0#046353AQ1=
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, QUOTRIX",,BF4SNM7,DEA19JW6=QTX
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, STUTTGART STOCK EXCHANGE",,BF4SNM7,DEA19JW6=SG
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEECHO APA EU",,BF4SNM7,US046353AQ14=TRU
"NT, Note, US046353AQ14 MiFID II Fine Grained RIC, Marketaxess Europe Limited",,,US046353AQ14=MAXM
"NT, Note, US046353AQ14 MiFID II Fine Grained RIC, Deutsche Boerse AG APA Service",,,US046353AQ14=DAp
"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,


#### Common Properties - RCS (Reuters Classification Scheme)
Throughout many documents, you will notice a number of RCS fields that are popular when selecting and filtering.

*Note* - I'll use navigators to demonstrate the values of these common properties.

In [15]:
// RCSAssetCategoryLeaf
// Specific to assets, this property will categorize the results into a readable identifier such as: 'Bond', 
// 'Commodity Future', 'Commercial Paper', etc.  Where the property SearchAllCategoryv3 provides general categories, 
// this property will be more specific to assets.  Pull out the top 10.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSAssetCategoryLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"]);

[
  {
    "Label": "Bond",
    "Count": 18203056
  },
  {
    "Label": "Barrier Warrant",
    "Count": 12762130
  },
  {
    "Label": "Commodity Future",
    "Count": 5706560
  },
  {
    "Label": "Traditional Warrant",
    "Count": 4827105
  },
  {
    "Label": "Equity Cash Option",
    "Count": 3856763
  },
  {
    "Label": "Interest Rate Future Spread",
    "Count": 3657518
  },
  {
    "Label": "Commodity Spread",
    "Count": 3082155
  },
  {
    "Label": "Equity Future",
    "Count": 2929529
  },
  {
    "Label": "Mortgage-Backed Security",
    "Count": 2894634
  },
  {
    "Label": "Certificate of Deposit",
    "Count": 2540188
  },
  {
    "Label": "Commercial Paper",
    "Count": 2366882
  },
  {
    "Label": "Commodity Spot",
    "Count": 2175925
  },
  {
    "Label": "Warrant",
    "Count": 2146053
  },
  {
    "Label": "Commodity Future Option",
    "Count": 2132175
  },
  {
    "Label": "Commodity Forward",
    "Count": 1947445
  },
  {
    "Label": "Ordinary Share",
    "

In [16]:
// RCSCountryLeaf
// RCSIssuerCountryLeaf
// RCSExchangeCountryLeaf
// Specific to some assets, will allow you to narrow your search to focus on a specific country.  Pull out the top 20.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSCountryLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSCountryLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 13669059
  },
  {
    "Label": "Eurobond",
    "Count": 4560770
  },
  {
    "Label": "China (Mainland)",
    "Count": 1289076
  },
  {
    "Label": "Colombia",
    "Count": 1093363
  },
  {
    "Label": "South Korea",
    "Count": 583887
  },
  {
    "Label": "Germany",
    "Count": 506187
  },
  {
    "Label": "Canada",
    "Count": 496159
  },
  {
    "Label": "France",
    "Count": 273195
  },
  {
    "Label": "India",
    "Count": 212293
  },
  {
    "Label": "Japan",
    "Count": 165382
  },
  {
    "Label": "Italy",
    "Count": 135288
  },
  {
    "Label": "Australia",
    "Count": 134035
  },
  {
    "Label": "Chile",
    "Count": 131629
  },
  {
    "Label": "Brazil",
    "Count": 82661
  },
  {
    "Label": "Switzerland",
    "Count": 72074
  },
  {
    "Label": "Austria",
    "Count": 60904
  },
  {
    "Label": "United Kingdom",
    "Count": 60278
  },
  {
    "Label": "Spain",
    "Count": 59978
  },
  {
    "Label": "Norwa

In [17]:
// RCSCurrencyLeaf
// Specific to some assets, will allow you to narrow your search to focus on a specific currency.  Just pull out the top 20.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSCurrencyLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSCurrencyLeaf"]["Buckets"]);

[
  {
    "Label": "US Dollar",
    "Count": 35821024
  },
  {
    "Label": "Euro",
    "Count": 30537519
  },
  {
    "Label": "Chinese Yuan",
    "Count": 2637091
  },
  {
    "Label": "US Cent",
    "Count": 1570272
  },
  {
    "Label": "Colombian Peso",
    "Count": 1305963
  },
  {
    "Label": "Japanese Yen",
    "Count": 1204079
  },
  {
    "Label": "Hong Kong Dollar",
    "Count": 1147187
  },
  {
    "Label": "Canadian Dollar",
    "Count": 1075591
  },
  {
    "Label": "Swedish Krona",
    "Count": 1012115
  },
  {
    "Label": "Indian Rupee",
    "Count": 918812
  },
  {
    "Label": "British Pound",
    "Count": 885963
  },
  {
    "Label": "Swiss Franc",
    "Count": 882080
  },
  {
    "Label": "South Korean Won",
    "Count": 818612
  },
  {
    "Label": "Australian Dollar",
    "Count": 562286
  },
  {
    "Label": "British Penny",
    "Count": 536812
  },
  {
    "Label": "Hungarian Forint",
    "Count": 435014
  },
  {
    "Label": "Norwegian Krone",
    "Count": 43

In [18]:
// RCSTRBC2012Leaf
// Represents the industry sector.  For example, if I query for Apple Computers, I will get many different 
// industry sectors such as: 'Computer Hardware (NEC)', 'Laptop & Desktop Computers', 'Phones & Smart Phones', etc.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSTRBC2012Leaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSTRBC2012Leaf"]["Buckets"]);

[
  {
    "Label": "Banks (NEC)",
    "Count": 4823853
  },
  {
    "Label": "Corporate Financial Services (NEC)",
    "Count": 2231491
  },
  {
    "Label": "Retail & Mortgage Banks",
    "Count": 2078407
  },
  {
    "Label": "Corporate Banks",
    "Count": 1508932
  },
  {
    "Label": "Investment Banking & Brokerage Services (NEC)",
    "Count": 823373
  },
  {
    "Label": "Government & Government Finance (NEC)",
    "Count": 670407
  },
  {
    "Label": "Public Finance Activities",
    "Count": 531455
  },
  {
    "Label": "Investment Holding Companies (NEC)",
    "Count": 498895
  },
  {
    "Label": "Construction & Engineering (NEC)",
    "Count": 420469
  },
  {
    "Label": "Consumer Lending (NEC)",
    "Count": 379158
  },
  {
    "Label": "Electric Utilities (NEC)",
    "Count": 364255
  },
  {
    "Label": "Investment Banking",
    "Count": 357647
  },
  {
    "Label": "Business Support Services (NEC)",
    "Count": 348495
  },
  {
    "Label": "Wealth Management",
    "Co

In [19]:
// RCSTRBC2012Leaf
// RCSTRBC2012Genealogy
// RCSTRBC2012Name
// Provide a breakdown of the hierarchy defined for many of these RCS attributes.  You can see the mapping between the 
// hierarchy defined with the Genealogy property and Name property.  
// For example, the code 'B:262' maps to the name 'Banking & Investment Services'.
response = Search.Definition(Search.View.SearchAll).Filter("RCSTRBC2012Leaf eq 'Banks'")
                                                   .Top(0)
                                                   .Navigators("RCSTRBC2012Leaf, RCSTRBC2012Genealogy, RCSTRBC2012Name")
                                                   .GetData();
Console.WriteLine(response.Data.Raw);

{
  "Total": 8459292,
  "Hits": [],
  "Navigators": {
    "RCSTRBC2012Genealogy": {
      "Buckets": [
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1615",
          "Count": 4823853
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1617",
          "Count": 2078469
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1616",
          "Count": 1508932
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1619",
          "Count": 23706
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:131\\B:1629",
          "Count": 21862
        },
        {
          "Label": "B:261\\B:262\\B:263\\B:207\\B:1638",
          "Count": 1178
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1620",
          "Count": 1064
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1618",
          "Count": 228
        }
      ]
    },
    "RCSTRBC2012Leaf": {
      "Buckets": [
        {
   

The above output includes 3 separate sections: RCSTRBC2012Genealogy, RCSTRBC2012Leaf and RCSTRBC2012Name.  For each you can see a breakdown ot the buckets.